In [ ]:
import pandas
import bokeh
from bokeh.io import show
from bokeh.io import output_notebook
from bokeh.models import EqHistColorMapper
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

bokeh.sampledata.download()
from bokeh.sampledata.us_counties import data as counties

In [ ]:
autism = pandas.read_csv("ADV_AllData.csv")
state_list = [s.lower() for s in set(autism['Location'])]
state_list.remove('ak')
state_list.remove('hi')

census = pandas.read_csv("census.txt", dtype={'year':int, 'fips':str, 'age':int, 'pop':int})

In [ ]:
counties = {
    code: county for code, county in counties.items() if county["state"] in state_list
}
county_pop = {}
for i in range(2):
    census_filt = census[census['year']==2000+i]
    county_pop[2000+i] = [float(census_filt[census_filt['fips']=="%02d"%(county_id[0])+"%03d"%(county_id[1])]['pop'].sum()) for county_id in counties]

In [ ]:
output_notebook()
palette = tuple(reversed(palette))



county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]

county_names = [county['name'] for county in counties.values()]

color_mapper = EqHistColorMapper(palette=palette)

data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
)
for i in range(2):
    data.update({"pop%d"%(2000+i):county_pop[2000+i]})

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(plot_width=1000, plot_height=800,
    title="USA", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    tooltips=[
        ("Name", "@name"), ("Population", "@pop2000"), ("(Long, Lat)", "($x, $y)")
    ])
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('x', 'y', source=data,
          fill_color={'field': 'pop2000', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

show(p)